In [3]:
import pandas as pd

In [4]:
path_death = 'causes_of_death.csv'

df_death = pd.read_csv(path_death)
df_death.head(2)

,Year,ZIP Code,Causes of Death,Count,Location
0,1999,90002,SUI,1,"(33.94969, -118.246213)"
1,1999,90005,HOM,1,"(34.058508, -118.301197)"


In [5]:
path_income = 'irs_income_by_zip_code.csv'

df_income = pd.read_csv(path_income)
df_income.head(2)

,STATE,ZIPCODE,Number of returns,Adjusted gross income (AGI),Avg AGI,Number of returns with total income,Total income amount,Avg total income,Number of returns with taxable income,Taxable income amount,Avg taxable income
0,AL,0,2022380,105089761,51.963410,2022380,106420533,52.621433,1468370,67850874,46.208295
1,AL,35004,4930,255534,51.832454,4930,258024,52.337525,4020,163859,40.760945


In [11]:
df_death_final = df_death[['ZIP Code','Causes of Death','Count']]
df_death_final.head(2)

,ZIP Code,Causes of Death,Count
0,90002,SUI,1
1,90005,HOM,1


In [13]:
df_income_final = df_income[['ZIPCODE', 'Avg AGI']]
df_income_final.head(2)

,ZIPCODE,Avg AGI
0,0,51.963410
1,35004,51.832454
